<a href="https://colab.research.google.com/github/ChavChavC/BT4222/blob/main/FE_NGram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 6.6 MB/s eta 0:00:00


# NGRAM

In [39]:
import pandas as pd

df = pd.read_csv('try_cleaned_data.csv')
df = df.drop(columns=df.columns[0], axis=1)
df[df.isnull().any(axis=1)]
df.dropna(subset=['title'], inplace=True)
data= df['title'].tolist()
labels_df = df[['labels']]
labels_array = df['labels'].tolist()

In [ ]:
#check for foreign characters
df.loc[df['title'].str.contains('気候の非常事態への対策として大幅な炭素削減を可能にするモバイル技術')]

,title,labels


In [ ]:
# tfidf generator
from sklearn.feature_extraction.text import TfidfVectorizer

# For only bigrams
#bigram_vectorizer = TfidfVectorizer(ngram_range=(2, 2))
#bigram_matrix = bigram_vectorizer.fit_transform(data)
#dense_matrix = bigram_matrix.todense()

# Get feature names to see which bigrams are in the vocabulary
#feature_names = bigram_vectorizer.get_feature_names_out()
#bigram_df = pd.DataFrame(bigram_matrix.toarray(), columns=feature_names)

# For both unigrams and bigrams
ngram_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
ngram_matrix = ngram_vectorizer.fit_transform(data)
ngram_dense_matrix = ngram_matrix.todense()

# Get feature names to see which bigrams are in the vocabulary
feature_names = ngram_vectorizer.get_feature_names_out()

# Display the matrix or feature names
# print(dense_matrix)
#print(len(feature_names))
ngram_df = pd.DataFrame(ngram_matrix.toarray(), columns=feature_names)

In [ ]:
ngram_df

Top K Ngrams as features using RandomForest clf

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(ngram_matrix, labels_array)  # 'labels' is the array of sentiment labels

importances = clf.feature_importances_

In [ ]:
import numpy as np
# Get indices of sorted importances
sorted_indices = np.argsort(importances)[::-1]

# Select the top N feature indices
N = 1000  # For example, select the top 1000 features
top_feature_indices = sorted_indices[:N]

# Select the top N features from the original feature matrix
X_top_ngrams = ngram_matrix[:, top_feature_indices]

NER experimentation

In [ ]:
!pip install spacy transformers

In [40]:
# try word capitalization to improve results (failed)
#df['title'] = df['title'].str.lower().apply(string.capwords)

In [116]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import spacy, string

nlp = spacy.load("en_core_web_sm")
def perform_ner(text):
    doc = nlp(text)
    organizations = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    return organizations

df['NER'] = df['title'].apply(perform_ner)

def refine_org_name(ner_results):
    orgs_list = []
    org_name = ""
    # A set of common words not typically part of an organization name
    non_org_keywords = {'reports', 'results', 'quarter', 'earnings', 'profit', 'loss', 'revenue', 'confirms', 'increase', 'delivers'}

    for result in ner_results:

        entity = result['entity']
        word = result['word']

        # check for first org occurence
        if entity == 'B-ORG':
          #add prev org to list
          if len(org_name) > 0:
            orgs_list.append(org_name.strip())
          org_name = word
        # check for I org subword
        elif entity == 'I-ORG':
          if word[0:2] == "##":
            word = word.replace('##', '')
            org_name += word
        # check for I org but not subword
          elif(word.lower() not in non_org_keywords):
            org_name += f" {word}"

    if org_name.strip() not in orgs_list and org_name:
      orgs_list.append(org_name.strip())

    return orgs_list


# Use the refined function to reconstruct the organization name
def BERT_NER(dataset):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  NER_res = []

  for test in tests:
    NER_res.append(refine_org_name(nlp(test)))

  return NER_res


In [109]:
spacy_results = df.NER[0:50].tolist()
spacy_results

[[],
 ['TRILLION ENERGY ANNOUNCES FLOW TEST RESULTS FOR WELL'],
 [],
 ['Unigold Inc Delivers Positive Feasibility Study for Candelones Oxide Project'],
 [],
 ['Ascendant Resources Reports Copper Intercept', 'Lagoa Salgada'],
 ['Barrick'],
 ['TomaGold'],
 ['Surge Energy Inc Announces',
  'Financings Anticipated Dividend Increase Increased Production Rate'],
 ['Stack Capital Group Inc'],
 ['Diversified Royalty Corp Completes Acquisition', 'Confirms Increase'],
 ['Visionary Education Technology', 'LOI', 'Acquire Equity Interest'],
 ['Xenon Pharmaceuticals Announces Conference Call and Webcast to Discuss Third Quarter Financial Results'],
 ['InMed Pharmaceuticals Advances Neurodegenerative Disease Program',
  'Natural Sciences and Engineering Research Council of Canada NSERC Alliance'],
 ['Acquire Additional Mining', 'Exclusive Prospective Licences'],
 ['Canadian Life Companies Split Corp Preferred'],
 [],
 ['Ceapro Inc Launches Next Phase', 'Disruptive PGX Technology'],
 ['TELUS Internati

In [115]:
# 34s for 50 entries, 40 mins for 5000
tests = df.title[0:50].tolist()
bert_results = BERT_NER(tests)
bert_results

Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# captures orgs not in bert but in spacy but spacy inaccurate sometimes (useless)
def combine_results(spacy, bert):

    combined_results = []
    for orgs_spacy, orgs_bert in zip(spacy, bert):
        #check both items are empty or not
        if orgs_spacy or orgs_bert:

            #check if either is empty
            if len(orgs_spacy) == 0 and orgs_bert:
              combined_results.append(orgs_bert)
            elif len(orgs_bert) == 0 and orgs_spacy:
              combined_results.append(orgs_spacy)
            else:
              #favour bert so set bert as default
              result = orgs_bert

              # check for any org overlap
              for org_spacy in orgs_spacy:
                overlap = False

                if org_spacy in orgs_bert:
                  # dont add this org
                  continue

                else:
                  for subword_org_spacy in org_spacy.split():
                    if subword_org_spacy in ' '.join(orgs_bert).split():
                      overlap = True
                      break
                  if not overlap:
                    result.append(org_spacy)
              combined_results.append(result)
        else:
          combined_results.append('No Entity Recognised')

    # Return the combined list of results
    return combined_results

test1 = combine_results(spacy_results,bert_results)

def combine_results(spacy, bert):
    combined_results = []
    for orgs_spacy, orgs_bert in zip(spacy, bert):
      if orgs_bert:
        combined_results.append(orgs_bert)
      elif orgs_spacy:
        combined_results.append(orgs_spacy)
      else:
        combined_results.append('No Entity Recognised')
    return combined_results

test2 = combine_results(spacy_results, bert_results)

for result1, result2 in zip(test1, test2):
  print(result1)
  print(result2)
  print('----\n')


# FINAL NER CODE TO USE

Load Dataset

In [136]:
import pandas as pd

df = pd.read_csv('try_cleaned_data.csv')
df = df.drop(columns=df.columns[0], axis=1)
df[df.isnull().any(axis=1)]
df.dropna(subset=['title'], inplace=True)
data= df['title'].tolist()
labels_df = df[['labels']]
labels_array = df['labels'].tolist()

SPACY NER

In [ ]:
# slice and tried using top 200 rows
nlp = spacy.load("en_core_web_sm")
def perform_ner(text):
    doc = nlp(text)
    organizations = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    return organizations

df['NER'] = df['title'].apply(perform_ner)
spacy_results = df.NER[0:200].tolist()
spacy_results

BERT NER

In [ ]:

# slice and tried using top 200 rows
def refine_org_name(ner_results):
    orgs_list = []
    org_name = ""
    # A set of common words not typically part of an organization name
    non_org_keywords = {'reports', 'results', 'quarter', 'earnings', 'profit', 'loss', 'revenue', 'confirms', 'increase', 'delivers'}

    for result in ner_results:

        entity = result['entity']
        word = result['word']

        # check for first org occurence
        subword_flag = False
        if entity == 'B-ORG':
          # check if subword
          if word[0:2] == "##":
            word = word.replace('##', '')
            org_name += word
            subword_flag = True
          else:
            #add prev org to list
            if len(org_name) > 0:
              orgs_list.append(org_name.strip())
            org_name = word
        # check for I org subword
        elif entity == 'I-ORG':
          if word[0:2] == "##":
            word = word.replace('##', '')
            org_name += word
        # check for I org but not subword
          elif(word.lower() not in non_org_keywords):
            org_name += f" {word}"

    if org_name.strip() not in orgs_list and org_name:
      orgs_list.append(org_name.strip())

    return orgs_list


# Use the refined function to reconstruct the organization name
def BERT_NER(dataset):
  tokenizer = AutoTokenizer.from_pretrained("butchland/bert-finetuned-ner")
  model = AutoModelForTokenClassification.from_pretrained("butchland/bert-finetuned-ner")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer, device=0 )
  NER_res = []

  for sentence in dataset:
    sentence_orgs = refine_org_name(nlp(sentence))
    # one more layer of check if ORG name not in original sentence
    # handling unexpected orgs like dividend -> dind
    # for org in sentence_orgs:
    #   for word in org.split():
    #     if word not in sentence.split():
    #       print(word)
    #       print(sentence)
    #       print(sentence_orgs)
    #       sentence_orgs.remove(org)
    NER_res.append(sentence_orgs)
  return NER_res

tests = df.title[0:200].tolist()
bert_results = BERT_NER(tests)
bert_results


##Combine SPACY and BERT NER Results

In [139]:
def combine_results(spacy, bert):
    combined_results = []
    for orgs_spacy, orgs_bert in zip(spacy, bert):
      if orgs_bert:
        combined_results.append(orgs_bert)
      elif orgs_spacy:
        combined_results.append(orgs_spacy)
      else:
        combined_results.append('No Entity Recognised')
    return combined_results

NER_col_results = combine_results(spacy_results, bert_results)
# for i in range(13684-50):
#   NER_col_results.append('NA')
#df = df.loc[0:199]
df['Entities'] = NER_col_results

<ipython-input-139-4999c462c043>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Entities'] = NER_col_results
